In [1]:
import random
from gophish import Gophish
from gophish.models import SMTP, Page, Template, Campaign, User, Group
import urllib3
import json

In [2]:
config_file = "Alek_config.json"

with open(config_file, "r") as file:
    config = json.load(file)
    GOPHISH_API_KEY = config.get("GOPHISH_API_KEY")
    gmail_pass = config.get("gmail_pass")

In [3]:
phish_api = Gophish(GOPHISH_API_KEY, verify=False)

In [50]:
# SMTP Sending Profile Data
sending_profile = SMTP(
    name="Document Sign",
    from_address="aleksander.szostakowski@gmail.com",  # Replace with your email address
    from_name="Docusign",
    reply_to_address="aleksander.szostakowski@gmail.com",  # Replace with your email address
    host="smtp.gmail.com:587",  # Replace with your SMTP server host
    username="aleksander.szostakowski@gmail.com",  # Replace with your SMTP username
    password=gmail_pass,  # Replace with your SMTP password
    port=587,  # SMTP port (use 465 for SSL, 587 for TLS)
    tls=True,  # Use TLS if needed
    ssl=False,  # Use SSL if needed
)


In [51]:
smtp = phish_api.smtp.post(sending_profile)

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
for mod in phish_api.smtp.get():
    print(mod.__dict__)

{'id': 2, 'interface_type': 'SMTP', 'name': 'Training Sending Profile', 'host': 'smtp.gmail.com:587', 'username': 'aleksander.szostakowski@gmail.com', 'password': 'icil uzig kfil fjba', 'from_address': 'aleksander.szostakowski@gmail.com', 'ignore_cert_errors': False, 'modified_date': datetime.datetime(2025, 1, 15, 13, 54, 40, 328839, tzinfo=tzutc()), 'headers': []}
{'id': 3, 'interface_type': 'SMTP', 'name': 'Document Sign', 'host': 'smtp.gmail.com:587', 'username': 'aleksander.szostakowski@gmail.com', 'password': 'icil uzig kfil fjba', 'from_address': 'aleksander.szostakowski@gmail.com', 'ignore_cert_errors': False, 'modified_date': datetime.datetime(2025, 1, 16, 13, 54, 20, 361290, tzinfo=tzutc()), 'headers': []}


c:\Python311\Lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [68]:
with open("../web-pages/index.html", "r") as file:
    dummy_html = file.read()


In [69]:
landing_page = Page(
            name="Document Sign Page",
            url="127.0.0.1:3333",
            html=dummy_html,
            capture_credentials=True,
            capture_passwords=True
        )

In [70]:
phish_api.pages.post(landing_page)

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [71]:
with open('docu_sign_mail.html') as file:
    dummy_mail = file.read()


In [72]:
email_template = Template(
    name='Docu Sign Scam',
    subject='Sign Company Documents',
    html=dummy_mail,    
)

In [73]:
phish_api.templates.post(email_template)

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [42]:
group_2 = Group(name='Example Gr', targets=[User(first_name='Alek', last_name='Szostakowski', email='aleksander.szostakowski@gmail.com', posistion='data janitor')])

In [43]:
phish_api.groups.post(group_2)

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Error: Group name already in use

In [75]:
campaign = Campaign(
    name="Document Sign Scam",
    template=Template(name="Docu Sign Scam"),
    page = Page(name="Document Sign Page"),
    smtp=SMTP(name="Document Sign"),
    groups=[Group(name="Example Gr")],
    url="http://127.0.0.1"
)

In [76]:
phish_api.campaigns.post(campaign)

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [35]:
campaign.__dict__

{'id': None,
 'name': 'Document Sign Scam',
 'created_date': datetime.datetime(2025, 1, 17, 9, 39, 1, 790852, tzinfo=tzlocal()),
 'launch_date': datetime.datetime(2025, 1, 17, 9, 39, 1, 790852, tzinfo=tzlocal()),
 'send_by_date': None,
 'completed_date': None,
 'template': <gophish.models.Template at 0x25a15905490>,
 'page': <gophish.models.Page at 0x25a15979750>,
 'results': [],
 'status': None,
 'timeline': [],
 'smtp': <gophish.models.SMTP at 0x25a15af8950>,
 'url': None,
 'groups': [{'name': 'Example Gr',
   'modified_date': '2025-01-17T09:39:01.791356+01:00'}]}

In [62]:
camp = phish_api.campaigns.get(3)
camp.__dict__

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'id': 3,
 'name': 'Cybersecurity Awareness Scam',
 'created_date': '2025-01-15T14:58:31.8999001Z',
 'launch_date': '2025-01-15T13:20:27.411353Z',
 'send_by_date': '0001-01-01T00:00:00Z',
 'completed_date': '0001-01-01T00:00:00Z',
 'template': <gophish.models.Template at 0x199bd2ffb50>,
 'page': <gophish.models.Page at 0x199bd179ed0>,
 'results': [<gophish.models.Result at 0x199bd2fda10>],
 'status': 'In progress',
 'timeline': [<gophish.models.TimelineEntry at 0x199bd2fc8d0>,
 'smtp': <gophish.models.SMTP at 0x199bd2ff5d0>,
 'url': '',
 'groups': []}

In [65]:
for x in camp.results:
    print(x.__dict__)

{'id': 'gVZGtHU', 'first_name': 'Aleksander', 'last_name': 'Szostakowski', 'email': 'aleksander.szostakowski@gmail.com', 'position': 'big boss', 'ip': '', 'latitude': 0, 'longitude': 0, 'status': 'Email Sent'}
